<a href="https://colab.research.google.com/github/DevSlap/M23COM_ArtificialNeuralNetwork/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Encoder decoder model for text summarisation**

## connect to google drive and map the path, so it can be accesed by jupyter notebook

In [0]:
!apt-get install software-properties-common
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15740102/+files/google-drive-ocamlfuse_0.7.1-0ubuntu3_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.1-0ubuntu3_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.96.24.32.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
E: Package 'python-software-properties' has no installation candidate
--2019-03-29 22:32:05--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15740102/+files/google-drive-ocamlfuse_0.7.1-0ubuntu3_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8003, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/400273474/google-drive-ocamlfuse_0.7.1-0ubuntu3_amd64.deb [following]
--2019-03-29 22:32:06--  https://laun

In [0]:
!pip install gensim
!pip install wget
  
import nltk
nltk.download('punkt')

  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec


In [0]:
default_path = "/content/drive/Colab Notebooks/Model 2/"

train_article_path = default_path + "sumdata/train/train.article.txt"
train_title_path   = default_path + "sumdata/train/train.title.txt"
valid_article_path = default_path + "sumdata/train/valid.article.filter.txt"
valid_title_path   = default_path + "sumdata/train/valid.title.filter.txt"


In [0]:

def clean_str(sentence):
    sentence = re.sub("[#.]+", "#", sentence)
    return sentence


def get_text_list(data_path, toy):
    with open (data_path, "r", encoding="utf-8") as f:
      return [clean_str(x.strip()) for x in f.readlines()][:toy]


In [0]:
def build_dict(step, toy=50):
    if step == "train":
        train_article_list = get_text_list(train_article_path, toy)
        train_title_list = get_text_list(train_title_path, toy)

        words = list()
        for sentence in train_article_list + train_title_list:
            for word in word_tokenize(sentence):
                words.append(word)

        word_counter = collections.Counter(words).most_common()
        word_dict = dict()
        word_dict["<padding>"] = 0
        word_dict["<unk>"] = 1
        word_dict["<s>"] = 2
        word_dict["</s>"] = 3
        for word, _ in word_counter:
            word_dict[word] = len(word_dict)

        with open(default_path + "word_dict.pickle", "wb") as f:
            pickle.dump(word_dict, f)

    elif step == "valid":
        with open(default_path + "word_dict.pickle", "rb") as f:
            word_dict = pickle.load(f)

    reversed_dict = dict(zip(word_dict.values(), word_dict.keys()))

    article_max_len = 100
    summary_max_len = 15
#   summary_max_len = 15

    return word_dict, reversed_dict, article_max_len, summary_max_len

In [0]:
def build_dataset(step, word_dict, article_max_len, summary_max_len, toy=50):
    if step == "train":
        article_list = get_text_list(train_article_path, toy)
        title_list = get_text_list(train_title_path, toy)
    elif step == "valid":
        article_list = get_text_list(valid_article_path, toy)
    else:
        raise NotImplementedError

    x = [word_tokenize(d) for d in article_list]
    x = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in x]
    x = [d[:article_max_len] for d in x]
    x = [d + (article_max_len - len(d)) * [word_dict["<padding>"]] for d in x]
    
    if step == "valid":
        return x
    else:        
        y = [word_tokenize(d) for d in title_list]
        y = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in y]
        y = [d[:(summary_max_len - 1)] for d in y]
        return x, y



In [0]:
def batch_iter(inputs, outputs, batch_size, num_epochs):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]


In [0]:
def get_init_embedding(reversed_dict, embedding_size):
    
    print("Loading Glove vectors...")
   

    with open( default_path + "glove/model_glove_300.pkl", 'rb') as handle:
        word_vectors = pickle.load(handle)
        
    word_vec_list = list()
    for _, word in sorted(reversed_dict.items()):
        try:
            word_vec = word_vectors.word_vec(word)
        except KeyError:
            word_vec = np.zeros([embedding_size], dtype=np.float32)

        word_vec_list.append(word_vec)

    # Assign random vector to <s>, </s> token
    word_vec_list[2] = np.random.normal(0, 1, embedding_size)
    word_vec_list[3] = np.random.normal(0, 1, embedding_size)

    return np.array(word_vec_list)

In [0]:
import tensorflow as tf
from tensorflow.contrib import rnn



class Model(object):
    def __init__(self, reversed_dict, article_max_len, summary_max_len, args, forward_only=False):
        self.vocabulary_size = len(reversed_dict)
        self.embedding_size = args.embedding_size
        self.num_hidden = args.num_hidden
        self.num_layers = args.num_layers
        self.learning_rate = args.learning_rate
        self.beam_width = args.beam_width
        if not forward_only:
            self.keep_prob = args.keep_prob
        else:
            self.keep_prob = 1.0
        self.cell = tf.nn.rnn_cell.BasicLSTMCell
        with tf.variable_scope("decoder/projection"):
            self.projection_layer = tf.layers.Dense(self.vocabulary_size, use_bias=False)
        print('vocabulary size {0}'.format(self.vocabulary_size))
        self.batch_size = tf.placeholder(tf.int32, (), name="batch_size")
        self.X = tf.placeholder(tf.int32, [None, article_max_len])
        self.X_len = tf.placeholder(tf.int32, [None])
        self.decoder_input = tf.placeholder(tf.int32, [None, summary_max_len])
        self.decoder_len = tf.placeholder(tf.int32, [None])
        self.decoder_target = tf.placeholder(tf.int32, [None, summary_max_len])
        self.global_step = tf.Variable(0, trainable=False)

        with tf.name_scope("embedding"):
            if not forward_only and args.glove:
   
                init_embeddings = tf.constant(get_init_embedding(reversed_dict, self.embedding_size), dtype=tf.float32)
            else:
                init_embeddings = tf.random_uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0)
            self.embeddings = tf.get_variable("embeddings", initializer=init_embeddings)
            self.encoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.X), perm=[1, 0, 2])
            self.decoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.decoder_input), perm=[1, 0, 2])

        with tf.name_scope("encoder"):
            fw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            bw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            fw_cells = [rnn.DropoutWrapper(cell) for cell in fw_cells]
            bw_cells = [rnn.DropoutWrapper(cell) for cell in bw_cells]

            encoder_outputs, encoder_state_fw, encoder_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                fw_cells, bw_cells, self.encoder_emb_inp,
                sequence_length=self.X_len, time_major=True, dtype=tf.float32)
            self.encoder_output = tf.concat(encoder_outputs, 2)
            encoder_state_c = tf.concat((encoder_state_fw[0].c, encoder_state_bw[0].c), 1)
            encoder_state_h = tf.concat((encoder_state_fw[0].h, encoder_state_bw[0].h), 1)
            self.encoder_state = rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)


        with tf.name_scope("decoder"), tf.variable_scope("decoder") as decoder_scope:
            decoder_cell = self.cell(self.num_hidden * 2)

            if not forward_only:
                attention_states = tf.transpose(self.encoder_output, [1, 0, 2])
            
                if args.attention_type in 'BahdanauAttention':
                  
                  attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    self.num_hidden * 2, attention_states, memory_sequence_length=self.X_len, normalize=True)
              
                  decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                  initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size)               
                  initial_state = initial_state.clone(cell_state=self.encoder_state)
                else:
                  

                  attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                    self.num_hidden * 2, attention_states, memory_sequence_length=self.X_len, scale=False)
              
                  decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                  initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size)               
                  initial_state = initial_state.clone(cell_state=self.encoder_state)
                
                
                helper = tf.contrib.seq2seq.TrainingHelper(self.decoder_emb_inp, self.decoder_len, time_major=True)
                decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=True, scope=decoder_scope)
                self.decoder_output = outputs.rnn_output
                self.logits = tf.transpose(
                    self.projection_layer(self.decoder_output), perm=[1, 0, 2])
                self.logits_reshape = tf.concat(
                    [self.logits, tf.zeros([self.batch_size, summary_max_len - tf.shape(self.logits)[1], self.vocabulary_size])], axis=1)
            else:
                tiled_encoder_output = tf.contrib.seq2seq.tile_batch(
                    tf.transpose(self.encoder_output, perm=[1, 0, 2]), multiplier=self.beam_width)
                tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(self.encoder_state, multiplier=self.beam_width)
                tiled_seq_len = tf.contrib.seq2seq.tile_batch(self.X_len, multiplier=self.beam_width)
                
                # Building attention mechanism    
                if args.attention_type in 'BahdanauAttention':
                  # 'Bahdanau' style attention: https://arxiv.org/abs/1409.0473
                  
                  attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                      self.num_hidden * 2, tiled_encoder_output, memory_sequence_length=tiled_seq_len, normalize=True)
                  decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                  initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size * self.beam_width)
                  initial_state = initial_state.clone(cell_state=tiled_encoder_final_state)
                else:
                  # 'Luong' style attention: https://arxiv.org/abs/1508.04025
                  
                  attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                      self.num_hidden * 2, tiled_encoder_output, memory_sequence_length=tiled_seq_len, scale=False)
                  decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                  initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size * self.beam_width)
                  initial_state = initial_state.clone(cell_state=tiled_encoder_final_state)
 
              
                decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                    cell=decoder_cell,
                    embedding=self.embeddings,
                    start_tokens=tf.fill([self.batch_size], tf.constant(2)),
                    end_token=tf.constant(3),
                    initial_state=initial_state,
                    beam_width=self.beam_width,
                    output_layer=self.projection_layer
                )
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder, output_time_major=True, maximum_iterations=summary_max_len, scope=decoder_scope)
                self.prediction = tf.transpose(outputs.predicted_ids, perm=[1, 2, 0])

        with tf.name_scope("loss"):
            if not forward_only:
                crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=self.logits_reshape, labels=self.decoder_target)
                weights = tf.sequence_mask(self.decoder_len, summary_max_len, dtype=tf.float32)
                self.loss = tf.reduce_sum(crossent * weights / tf.to_float(self.batch_size))

                params = tf.trainable_variables()
                gradients = tf.gradients(self.loss, params)
                clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
                optimizer = tf.train.AdamOptimizer(self.learning_rate)
                self.update = optimizer.apply_gradients(zip(clipped_gradients, params), global_step=self.global_step)

Train the model


In [0]:
import time
start = time.perf_counter()
import tensorflow as tf
import argparse
import pickle
import os



class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove=True
args.embedding_size=300

args.learning_rate=1e-3
args.batch_size=64
args.num_epochs=35
args.keep_prob = 0.8

args.toy=200000
args.with_model= False
args.attention_type = 'BahdanauAttention'
# args.attention_type = 'LuongAttention'
args.model_name = 'Model2_new_1'


if not os.path.exists(default_path + "saved_model"):
    os.mkdir(default_path + "saved_model")
else:
    if args.with_model:
        old_model_checkpoint_path = open('/tmp/saved_model/checkpoint', 'r')
        old_model_checkpoint_path = "".join(["",old_model_checkpoint_path.read().splitlines()[0].split('"')[1] ])

print("Building dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("train", args.toy)
print("Loading training dataset...")
train_x, train_y = build_dataset("train", word_dict, article_max_len, summary_max_len, args.toy)

tf.reset_default_graph()

with tf.Session() as sess:
    model = Model(reversed_dict, article_max_len, summary_max_len, args)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep = 3)
    if 'old_model_checkpoint_path' in globals() and args.with_model:
        print("Continuing from previous trained model:" , old_model_checkpoint_path , "...")
        saver.restore(sess, old_model_checkpoint_path )

    batches = batch_iter(train_x, train_y, args.batch_size, args.num_epochs)
    num_batches_per_epoch = (len(train_x) - 1) // args.batch_size + 1

    print("\nIteration starts.")
    print("Number of batches per epoch :", num_batches_per_epoch)
    for batch_x, batch_y in batches:
        batch_x_len = list(map(lambda x: len([y for y in x if y != 0]), batch_x))
        batch_decoder_input = list(map(lambda x: [word_dict["<s>"]] + list(x), batch_y))
        batch_decoder_len = list(map(lambda x: len([y for y in x if y != 0]), batch_decoder_input))
        batch_decoder_output = list(map(lambda x: list(x) + [word_dict["</s>"]], batch_y))

        batch_decoder_input = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_input))
        batch_decoder_output = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_output))

        train_feed_dict = {
            model.batch_size: len(batch_x),
            model.X: batch_x,
            model.X_len: batch_x_len,
            model.decoder_input: batch_decoder_input,
            model.decoder_len: batch_decoder_len,
            model.decoder_target: batch_decoder_output
        }

        _, step, loss = sess.run([model.update, model.global_step, model.loss], feed_dict=train_feed_dict)

        if step % 1000 == 0:
            print("step {0}: loss = {1}".format(step, loss))

        if step % num_batches_per_epoch == 0:
            hours, rem = divmod(time.perf_counter() - start, 3600)
            minutes, seconds = divmod(rem, 60)
            saver.save(sess, "/tmp/saved_model/model.ckpt", global_step=step)
            print(" Epoch {0}: Model is saved.".format(step // num_batches_per_epoch),
            "Elapsed: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds) , "\n")
        

 Test the model

In [0]:
import tensorflow as tf
import pickle

tf.reset_default_graph()

class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove=True
args.embedding_size=300

args.learning_rate=1e-3
args.batch_size=64
args.num_epochs=35
args.keep_prob = 0.8

args.toy=200000
args.with_model= False
args.attention_type = 'BahdanauAttention'
# args.attention_type = 'LuongAttention'
args.model_name = 'Model2_new_1'


print("Loading dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
print("Loading validation dataset...")
valid_x = build_dataset("valid", word_dict, article_max_len, summary_max_len, args.toy)
valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

print("Loading article and reference...")
article = get_text_list(valid_article_path, args.toy)
# print(article)
reference = get_text_list(valid_title_path, args.toy)
# print(reference)

with tf.Session() as sess:
    print("Loading saved model...")
    model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True)
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state("/tmp/saved_model/")
    saver.restore(sess, ckpt.model_checkpoint_path)

    batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)
    summary_array = []
    print("Writing summaries to " + args.model_name + "result.txt..")
    for batch_x, _ in batches:
        batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]

        valid_feed_dict = {
            model.batch_size: len(batch_x),
            model.X: batch_x,
            model.X_len: batch_x_len,
        }

        prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
        prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
        

   
        with open(default_path + args.model_name+"result.txt", "a+") as f:
            for line in prediction_output:
                summary = list()
                for word in line:
                    if word == "</s>":
                        break
                    if word not in summary:
                        summary.append(word)
                summary_array.append(" ".join(summary))
                print(" ".join(summary), file=f)
    print('Printing prediction output...')
    print(summary_array)

    print('Summaries have been generated')

## Evaluate & write output

In [0]:
!pip install sumeval
!python -m spacy download en

    100% |████████████████████████████████| 51kB 2.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/f1/7c/18/177331e7a5154401ab5fec8f3c6d2062508c38d832aaa3d5e2
  Stored in directory: /root/.cache/pip/wheels/56/c0/fb/1c7f9b3a71f64cdf86291cc645596f71746807bf2f72b3c1dd
Successfully built sumeval sacrebleu

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
print(reference)

["injury leaves kwan 's olympic hopes in limbo", 'us business attacks tough immigration law', 'gm december sales fall # percent', 'thousands of croatians celebrate before world cup slalom', "laura bush <unk> rice to attend sirleaf 's inauguration in liberia", 'top republican lobbyist pleads guilty to florida fraud', 'somalia rivals in compromise on seat of government', 'portuguese airport workers strike could ground flights on friday', 'maniche renews partnership with mourinho', "hollywood shores up support for ocean 's thirteen", 'canada recommends avoiding travel to nepal', 'ford executive sees weaker us auto sales in #', "us welcomes chinese journalist 's release highlights freedom concerns", 'two bankers admit theft from city players', 'unwell sharon admitted to hospital early', 'us hybrid vehicle sales expect to more than triple in seven years', "korea 's seo headed to dodgers from mets", 'hollywood starlet lindsay lohan admits bulimia battle', 'us insists soldiers act with restra

In [0]:

from sumeval.metrics.rouge import RougeCalculator


def eval_rouges(refrence_summary,model_summary):
    
    rouge = RougeCalculator(stopwords=True, lang="en")

    rouge_1 = rouge.rouge_n(
                summary=model_summary,
                references=refrence_summary,
                n=1)

    rouge_2 = rouge.rouge_n(
                summary=model_summary,
                references=[refrence_summary],
                n=2)
    
    rouge_l = rouge.rouge_l(
                summary=model_summary,
                references=[refrence_summary])
   
  
 
    
    
    return rouge_1, rouge_2,rouge_l
  


In [0]:
rouge_1_arr  = []
rouge_2_arr  = []
rouge_L_arr  = []

from xml.etree import ElementTree
from xml.dom import minidom
from functools import reduce

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")
  
from xml.etree.ElementTree import Element, SubElement, Comment

top = Element('Seq2Seq')

comment = Comment('Generated by Swapna Anandaraman')
top.append(comment)

i=0
for summ in summary_array:
  example = SubElement(top, 'example')
  article_element   = SubElement(example, 'article')
  article_element.text = article[i]
  
  reference_element = SubElement(example, 'gold_standard_summary')
  reference_element.text = reference[i]
  
  summary_element   = SubElement(example, 'generated_summary')
  summary_element.text = summ

  rouge_1, rouge_2,rouge_L = eval_rouges(reference[i],summ )
  
  eval_element = SubElement(example, 'eval')

  ROUGE_1_element  = SubElement(eval_element, 'ROUGE_1' , {'score':str(rouge_1)})
  ROUGE_2_element  = SubElement(eval_element, 'ROUGE_2' , {'score':str(rouge_2)})
  ROUGE_L_element  = SubElement(eval_element, 'ROUGE_L' , {'score':str(rouge_L)})
  
  

  rouge_1_arr.append(rouge_1) 
  rouge_2_arr.append(rouge_2) 
  rouge_L_arr.append(rouge_L) 
  
  i+=1
  


top.set('rouge_1', str(reduce(lambda x, y: x + y,  rouge_1_arr) / len(rouge_1_arr)))
top.set('rouge_2', str(reduce(lambda x, y: x + y,  rouge_2_arr) / len(rouge_2_arr)))
top.set('rouge_L', str(reduce(lambda x, y: x + y,  rouge_L_arr) / len(rouge_L_arr)))

with open(default_path + args.model_name + "result_xml.xml", "w+") as f:
  print(prettify(top), file=f)

In [0]:
# Calculate avg ROUGE values
rouge_1_sum = 0
rouge_2_sum = 0
rouge_L_sum = 0

i = 0
start = time.perf_counter()
for summ in summary_array:
  
  rouge_1, rouge_2,rouge_L = eval_rouges(reference[i],summ)
  rouge_1_sum = rouge_1_sum +  rouge_1
  rouge_2_sum = rouge_2_sum +  rouge_2
  rouge_L_sum = rouge_L_sum +  rouge_L
  i+= 1
  

  
summary_length =len(summary_array)
rouge_1_avg = rouge_1_sum/summary_length
rouge_2_avg = rouge_2_sum/summary_length
rouge_L_avg = rouge_L_sum/summary_length
  
  

# print
print('ROUGE 1:' + str(rouge_1_avg))
str(rouge_1_avg)
print('ROUGE 2:' + str(rouge_2_avg))
str(rouge_2_avg)
print('ROUGE L:' + str(rouge_L_avg))
str(rouge_L_avg)


hours, rem = divmod(time.perf_counter() - start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time taken for calculating ROUGE scores: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds) , "\n")

In [0]:
# print the parameters

print('Number of hidden nodes: '+ str(args.num_hidden))
print('Number of layers: ' + str(args.num_layers))
print('Beam Width: ' + str(args.beam_width))
print('Glove embedding used: ' + str(args.glove))
print('Embedding Dimension: ' + str(args.embedding_size))
print('Learning Rate: ' + str(args.learning_rate))
print('Batch Size: ' + str(args.batch_size))
print('Number of epochs: ' + str(args.num_epochs))
print('Attention type?: ' + str(args.attention_type))
print('ROUGE 1 ' + str(rouge_1_avg))
print('ROUGE 2 ' + str(rouge_2_avg))
print('ROUGE L ' + str(rouge_L_avg))



In [0]:
summary_array

['world champion kwan withdraws from skating championship',
 'us businessmen warn against use of illegal immigrants',
 'gm sales fall # percent in december',
 'croatian fans rally for world cup slalom',
 'laura bush rice secretary of state in food',
 'us lawmaker < unk > plead guilty in court to fraud case',
 'somalia parties agree compromise on transitional government',
 'portugal air strike planned',
 'kaka eyes portuguese loan signing',
 "ocean 's the movie is titanic #",
 'canada advises citizens to defer travel nepal',
 'auto sales to be weak in #',
 'us hails early release of chinese politician',
 'nigerian rugby workers admit fraud',
 'sharon admitted to hospital for',
 'us companies see triple sales of hybrid vehicles in',
 "dodgers acquire south korea 's < unk > fine",
 'asthma attack on the rise in britain',
 'bush insists us soldiers targeted in suicide bombing',
 'us envoy quits amid reports divisions over n # korea nuclear talks',
 'two tests positive for bird flu',
 'brit

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)